## Checking the backward phase

In [1]:
import numpy as np
import sempler
import sempler.generators
import sempler.utils
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
i = 0
path = 'causalicp/test/test_cases/case_%d' % i
XX = np.load(path + '.npy')
with open(path + '.pickle', 'rb') as f:
    (scm, target, interventions, data) = pickle.load(f)
assert (data == XX).all()

### Test ICP

In [3]:
import causalicp as icp

In [4]:
%%time
target = 0
result = icp.fit(XX, target, alpha=0.001, verbose=True)

Tested sets and their p-values
  set() accepted
  {1} accepted
  {2} rejected
  {3} rejected
  {4} accepted
  {1, 2} rejected
  {1, 3} rejected
  {1, 4} accepted
  {2, 3} rejected
  {2, 4} rejected
  {3, 4} rejected
  {1, 2, 3} rejected
  {1, 2, 4} rejected
  {1, 3, 4} rejected
  {2, 3, 4} rejected
  {1, 2, 3, 4} rejected

Estimated parental set: set()

CPU times: user 113 ms, sys: 2.61 ms, total: 115 ms
Wall time: 149 ms


In [5]:
for S in result.accepted:
    adapted = []
    for s in S:
        if s > target:
            adapted.append(s)
        else:
            adapted.append(s+1)
    print(adapted)

[]
[1]
[4]
[1, 4]


In [6]:
result.estimate

set()

### Test the GaussianData class

In [7]:
from causalicp import GaussianData

In [8]:
data = GaussianData(XX)
scm.W, scm.means

(array([[ 0.        ,  0.        ,  0.79620728, -0.52322521,  0.        ],
        [-0.        ,  0.        ,  0.84211651, -0.72007625,  0.        ],
        [-0.        ,  0.        ,  0.        , -0.62938999,  0.        ],
        [-0.        ,  0.        , -0.        , -0.        ,  0.        ],
        [ 0.        , -0.        ,  0.97444277,  0.        ,  0.        ]]),
 array([0.39488152, 0.29348817, 0.01407982, 0.1988424 , 0.71134195]))

#### Not the same to center data and to add intercept column

In [9]:
y = 2
S = {1,4}

In [10]:
data = GaussianData(XX, method = 'raw')

In [11]:
%%time
data.regress_pooled(y, S)

CPU times: user 744 µs, sys: 84 µs, total: 828 µs
Wall time: 664 µs


(array([0.        , 0.88039721, 0.        , 0.        , 1.01776983]),
 0.9552176802692507)

In [12]:
data = GaussianData(XX, method = 'scatter')

In [13]:
%%time
data.regress_pooled(y, S)

CPU times: user 255 µs, sys: 0 ns, total: 255 µs
Wall time: 237 µs


(array([0.        , 0.88039721, 0.        , 0.        , 1.01776983]),
 0.9552176802692511)